## Tools setup

In [ ]:
import os
import shutil

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!rm -r sample_data


In [ ]:
# to hide cell output
%%capture

!pip install tensorflow==2.*
#!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

!git clone https://github.com/tensorflow/models.git

# Compile protos.
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

%cd /content
!pip install cython

!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make

%cp -r pycocotools /content/models/research

%cd /content/models/research
%cp object_detection/packages/tf2/setup.py .
!python -m pip install .

# Test
#!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
workspace_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace'
models_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/models'
pre_trained_models_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models'
exported_models_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/exported_models'
tfrecord_output_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data'
train_record_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record'
validation_record_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record'
test_record_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/test.record'
label_map_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt'
dataset_path = '/content/drive/MyDrive/CVproject_Stenosis/Dataset'
images_path = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/Images'
csv_path = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'

object_detection_path = '/content/models/research/object_detection'
model_main_tf2 = 'model_main_tf2.py'
exporter_main_v2 = 'exporter_main_v2.py'

#copy_path1 = os.path.join(object_detection_path, model_main_tf2)
#copy_path2 = os.path.join(workspace_path, model_main_tf2)
#shutil.copy(copy_path1, copy_path2)

#copy_path1 = os.path.join(object_detection_path, exporter_main_v2)
#copy_path2 = os.path.join(workspace_path, exporter_main_v2)
#shutil.copy(copy_path1, copy_path2)

num_train_samples = 6700 # Sono 6692 in realtà ma mi sta comodo 6700 per fa i calcoli
num_eval_samples = 800
num_test_samples = 833

### Helper functions

In [ ]:
# Helper functions

def detect_fn(image):
    """
    Detect objects in image.
    
    Args:
      image: (tf.tensor): 4D input image
      
    Returs:
      detections (dict): predictions that model made
    """

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      numpy array with shape (img_height, img_width, 3)
    """
    
    return np.array(Image.open(path))


def nms(rects, thd=0.5):
    """
    Filter rectangles
    rects is array of oblects ([x1,y1,x2,y2], confidence, class)
    thd - intersection threshold (intersection divides min square of rectange)
    """
    out = []

    remove = [False] * len(rects)

    for i in range(0, len(rects) - 1):
        if remove[i]:
            continue
        inter = [0.0] * len(rects)
        for j in range(i, len(rects)):
            if remove[j]:
                continue
            inter[j] = intersection(rects[i][0], rects[j][0]) / min(square(rects[i][0]), square(rects[j][0]))

        max_prob = 0.0
        max_idx = 0
        for k in range(i, len(rects)):
            if inter[k] >= thd:
                if rects[k][1] > max_prob:
                    max_prob = rects[k][1]
                    max_idx = k

        for k in range(i, len(rects)):
            if (inter[k] >= thd) & (k != max_idx):
                remove[k] = True

    for k in range(0, len(rects)):
        if not remove[k]:
            out.append(rects[k])

    boxes = [box[0] for box in out]
    scores = [score[1] for score in out]
    classes = [cls[2] for cls in out]
    return boxes, scores, classes


def intersection(rect1, rect2):
    """
    Calculates square of intersection of two rectangles
    rect: list with coords of top-right and left-boom corners [x1,y1,x2,y2]
    return: square of intersection
    """
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]));
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]));
    overlapArea = x_overlap * y_overlap;
    return overlapArea


def square(rect):
    """
    Calculates square of rectangle
    """
    return abs(rect[2] - rect[0]) * abs(rect[3] - rect[1])


def inference_with_plot(path2images, draw_groundtruth=True, annotations_df=None, box_th=0.25):
    """
    Function that performs inference and plots resulting b-boxes
    
    Args:
      path2images: an array with pathes to images
      draw_groundtruth: if True will draw ground truth bbox, a pandas dataframe with annotations is needed
      annotations_dataframe: a pandas dataframe containing bbox annotations for ground truth
      box_th: (float) value that defines threshold for model prediction.
      
    Returns:
      None
    """
    for image_path in path2images:

        image_name = os.path.basename(image_path)
        print('Running inference for {}... '.format(image_name), end='')
        start_time = time.time()

        image_np = load_image_into_numpy_array(image_path)
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=200,
                min_score_thresh=box_th,
                agnostic_mode=False,
                line_thickness=3)

        #Code for drawing ground truth box
        if draw_groundtruth is True:
            xmin = annotations_df.loc[annotations_df['filename'] == image_name, 'xmin'].values[0]
            ymin = annotations_df.loc[annotations_df['filename'] == image_name, 'ymin'].values[0]
            xmax = annotations_df.loc[annotations_df['filename'] == image_name, 'xmax'].values[0]
            ymax = annotations_df.loc[annotations_df['filename'] == image_name, 'ymax'].values[0]

            viz_utils.draw_bounding_box_on_image_array(
                    image_np_with_detections,
                    ymin,
                    xmin,
                    ymax,
                    xmax,
                    use_normalized_coordinates=False,
                    display_str_list=['Real Stenosis'],
                    color='red',
                    thickness=2)
        
        plt.figure(figsize=(15,10))
        plt.imshow(image_np_with_detections)
        end_time = time.time() - start_time
        end_time = round(end_time, 2)
        print('Done.', 'took: ', end_time, ' s')
    plt.show()


def edit_configfile(conf_path, label_map_path, checkpoint_path, train_record_path, validation_record_path, batch_size, num_steps, is_ssdmodel=False):

    with open(conf_path, 'r') as f:
        config = f.read()

    with open(conf_path, 'w') as f:

        # Set labelmap path
        config = re.sub('label_map_path: ".*?"', 
                        'label_map_path: "{}"'.format(label_map_path), config)
    
        # Set fine_tune_checkpoint path
        config = re.sub('fine_tune_checkpoint: ".*?"',
                        'fine_tune_checkpoint: "{}"'.format(checkpoint_path), config)

        if is_ssdmodel is False:
        # Set train tf-record file path
            config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                            'input_path: "{}"'.format(train_record_path), config)

        else:
            config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 
                            'input_path: "{}"'.format(train_record_path), config, count=1)

        if is_ssdmodel is False:
            # Set validation tf-record file path FOR SSD MODEL
            config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                            'input_path: "{}"'.format(validation_record_path), config)

        else:
            # Set validation tf-record file path FOR SSD MODEL
            config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 
                            'input_path: "{}"'.format(validation_record_path), config, count=1)
    

        # Set number of classes.
        config = re.sub('num_classes: [0-9]+',
                    'num_classes: {}'.format(1), config)
    
        # Set batch size
        config = re.sub('batch_size: [0-9]+',
                        'batch_size: {}'.format(batch_size), config)
    
        # Set training steps
        config = re.sub('num_steps: [0-9]+',
                        'num_steps: {}'.format(num_steps), config)
        
        # Set optimizer steps
        config = re.sub('total_steps: [0-9]+',
                        'total_steps: {}'.format(num_steps), config)
    
        # Set fine-tune checkpoint type to detection
        config = re.sub('fine_tune_checkpoint_type: "classification"', 
                        'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
    
        f.write(config)


def normalize_bbox(bbox, rows, cols):
    """Normalize coordinates of a bounding box. Divide x-coordinates by image width and y-coordinates
    by image height.

    Args:
        bbox (tuple): Denormalized bounding box `(x_min, y_min, x_max, y_max)`.
        rows (int): Image height.
        cols (int): Image width.

    Returns:
        tuple: Normalized bounding box `(x_min, y_min, x_max, y_max)`.

    Raises:
        ValueError: If rows or cols is less or equal zero

    """
    (x_min, y_min, x_max, y_max), tail = bbox[:4], tuple(bbox[4:])

    if rows <= 0:
        raise ValueError("Argument rows must be positive integer")
    if cols <= 0:
        raise ValueError("Argument cols must be positive integer")

    x_min, x_max = x_min / cols, x_max / cols
    y_min, y_max = y_min / rows, y_max / rows

    return (x_min, y_min, x_max, y_max) + tail

## Utils

### Generazione dei TFrecord per train, test e validation

In [ ]:
# CSV path:      /content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV
# Images path:   /content/drive/MyDrive/CVproject_Stenosis/Dataset/Images
# TFrecord path: /content/drive/MyDrive/CVproject_Stenosis/workspace/data
# Gen TFrecord:  /content/drive/MyDrive/CVproject_Stenosis/workspace/generate_tfrecord.py

In [ ]:
'''
train_images_path = os.path.join(images_path, 'train')
train_csv_path = os.path.join(csv_path, 'train.csv')

!python /content/drive/MyDrive/CVproject_Stenosis/workspace/generate_tfrecord.py \
    --csv_input=$train_csv_path \
    --image_dir=$train_images_path \
    --output_path=$train_record_path
'''

In [ ]:
'''
test_images_path = os.path.join(images_path, 'test')
test_csv_path = os.path.join(csv_path, 'test.csv')

!python /content/drive/MyDrive/CVproject_Stenosis/workspace/generate_tfrecord.py \
    --csv_input=$test_csv_path \
    --image_dir=$test_images_path \
    --output_path=$test_record_path
'''

In [ ]:
'''
validation_images_path = os.path.join(images_path, 'validation')
validation_csv_path = os.path.join(csv_path, 'validation.csv')

!python /content/drive/MyDrive/CVproject_Stenosis/workspace/generate_tfrecord.py \
    --csv_input=$validation_csv_path \
    --image_dir=$validation_images_path \
    --output_path=$validation_record_path
'''

### Check vari sulle immagini e sui tfrecord

In [ ]:
from PIL import Image
import numpy as np

img = Image.open("/content/drive/MyDrive/CVproject_Stenosis/Dataset/Images/train/14_018_6_0017.bmp")
img_np = np.asarray(img)
print("img_np.shape: ", img_np.shape)

#<xmin>100</xmin><ymin>141</ymin><xmax>149</xmax><ymax>176</ymax>
tup = (100, 141, 149, 176)
normalize_bbox(tup, 512, 512)


In [ ]:
import tensorflow as tf 
raw_dataset1 = tf.data.TFRecordDataset("/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record")
raw_dataset2 = tf.data.TFRecordDataset("/content/drive/MyDrive/CVproject_Stenosis/workspace/data/test.record")
raw_dataset3 = tf.data.TFRecordDataset("/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")
print(raw_dataset1)
print(raw_dataset2)
print(raw_dataset3)

# SSD ResNet50 V1 FPN 640x640

## V1

### Configuration

In [ ]:
# ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
%rm ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

%cd ..

In [ ]:
model_tech_name = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
model_name = 'ssd_resnet50'
version_name = 'v1'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
# momentum_optimizer sembra essere il migliore in termini di loss
'''
model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    feature_extractor {
      type: "ssd_resnet50_v1_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 0.00039999998989515007
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 0.00039999998989515007
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.996999979019165
            scale: true
            epsilon: 0.0010000000474974513
          }
        }
        depth: 256
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 3.0
        aspect_ratios: [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.50, 2.75, 3.0]
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 2
  num_steps: 25200
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 13000
            learning_rate: 0.00003
          }
          schedule {
            step: 20000
            learning_rate: 0.000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  data_augmentation_options {
    random_horizontal_flip {
        probability: 0.5
    }
    random_pixel_value_scale {
        probability: 0.5
        minval: 0.9
        maxval: 1.1
    }
    random_adjust_brightness {
        probability: 0.5
        max_delta: 0.2
    }
    random_adjust_contrast {
        probability: 0.5
        min_delta: 0.8
        max_delta: 1.2
    }
  }

  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}

train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!cp $ckpt_data $ckpt_data2
!cp $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import time

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)


import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

## V2

### Configuration

In [ ]:
# ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
%rm ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

%cd ..

In [ ]:
model_tech_name = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
model_name = 'ssd_resnet50'
version_name = 'v2'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
# momentum_optimizer sembra essere il migliore in termini di loss
'''
model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    feature_extractor {
      type: "ssd_resnet50_v1_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 0.00039999998989515007
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        use_dropout: true
        dropout_keep_probability: 0.8
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 0.00039999998989515007
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.996999979019165
            scale: true
            epsilon: 0.0010000000474974513
          }
        }
        depth: 256
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 3.0
        aspect_ratios: [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.50, 2.75, 3.0]
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.1
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 2
  num_steps: 25200
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 13000
            learning_rate: 0.00003
          }
          schedule {
            step: 20000
            learning_rate: 0.000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  data_augmentation_options {
    random_horizontal_flip {
        probability: 0.5
    }
    random_pixel_value_scale {
        probability: 0.5
        minval: 0.9
        maxval: 1.3
    }
    random_adjust_brightness {
        probability: 0.5
        max_delta: 0.2
    }
    random_adjust_contrast {
        probability: 0.5
        min_delta: 0.8
        max_delta: 1.2
    }
  }

  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}

train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!cp $ckpt_data $ckpt_data2
!cp $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)


import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

# EfficientDet D0 512x512

## V1

### Configuration

In [ ]:
# efficientdet_d0_coco17_tpu-32

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
%rm efficientdet_d0_coco17_tpu-32.tar.gz

%cd ..

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v1'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
# momentum_optimizer sembra essere il migliore in termini di loss
'''
model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
          height: 512
          width: 512
      }
    }

    feature_extractor {
      type: "ssd_efficientnet-b0_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 3
        num_filters: 64
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        use_dropout: false
        dropout_keep_probability: 0.8
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.9999998989515007e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: SWISH
          batch_norm {
            decay: 0.9900000095367432
            scale: true
            epsilon: 0.0010000000474974513
          }
          force_use_bias: true
        }
        depth: 64
        num_layers_before_predictor: 3
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: [0.5, 1.0, 1.5, 2.0]
        scales_per_octave: 3
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 1.5
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 2.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    add_background_class: false
  }
}

train_config {
  batch_size: 4
#  freeze_variables: ["feature_extractor"]

  data_augmentation_options {
    random_horizontal_flip {
    }
    random_pixel_value_scale {

    }
    random_adjust_brightness {

    }
    random_adjust_contrast {
        
    }
  }

  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 15000
            learning_rate: 0.00003
          }
          schedule {
            step: 20000
            learning_rate: 0.00001
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"
  num_steps: 25200
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}
train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!mv $ckpt_data $ckpt_data2
!mv $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)


import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

## V2

### Configuration

In [ ]:
# efficientdet_d0_coco17_tpu-32

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
%rm efficientdet_d0_coco17_tpu-32.tar.gz

%cd ..

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v2'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
# momentum_optimizer sembra essere il migliore in termini di loss
'''
model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
          height: 512
          width: 512
      }
    }

    feature_extractor {
      type: "ssd_efficientnet-b0_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 3
        num_filters: 64
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        use_dropout: true
        dropout_keep_probability: 0.8
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.9999998989515007e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: SWISH
          batch_norm {
            decay: 0.9900000095367432
            scale: true
            epsilon: 0.0010000000474974513
          }
          force_use_bias: true
        }
        depth: 64
        num_layers_before_predictor: 3
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 3.0
        aspect_ratios: [0.5, 1.0, 1.5, 2.0]
        scales_per_octave: 3
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 1.5
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 2.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    add_background_class: false
  }
}

train_config {
  batch_size: 4
#  freeze_variables: ["feature_extractor"]

  data_augmentation_options {
    random_horizontal_flip {
    }
    random_pixel_value_scale {

    }
    random_adjust_brightness {

    }
    random_adjust_contrast {
        
    }
  }

  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 15000
            learning_rate: 0.00003
          }
          schedule {
            step: 20000
            learning_rate: 0.00001
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"
  num_steps: 25200
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}
train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!mv $ckpt_data $ckpt_data2
!mv $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)


import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

## V3

### Configuration

In [ ]:
# efficientdet_d0_coco17_tpu-32

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
%rm efficientdet_d0_coco17_tpu-32.tar.gz

%cd ..

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v3'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
# momentum_optimizer sembra essere il migliore in termini di loss
'''
model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
          height: 512
          width: 512
      }
    }

    feature_extractor {
      type: "ssd_efficientnet-b0_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 3
        num_filters: 64
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        use_dropout: true
        dropout_keep_probability: 0.8
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.9999998989515007e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: SWISH
          batch_norm {
            decay: 0.9900000095367432
            scale: true
            epsilon: 0.0010000000474974513
          }
          force_use_bias: true
        }
        depth: 64
        num_layers_before_predictor: 3
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 2.0
        aspect_ratios: [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.50, 2.75, 3.0]
        scales_per_octave: 3
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 1.5
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 2.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    add_background_class: false
  }
}

train_config {
  batch_size: 4
#  freeze_variables: ["feature_extractor"]
  data_augmentation_options {
    random_horizontal_flip {
        probability: 0.5
    }
    random_pixel_value_scale {
        probability: 0.5
        minval: 0.9
        maxval: 1.1
    }
    random_adjust_brightness {
        probability: 0.5
        max_delta: 0.2
    }
    random_adjust_contrast {
        probability: 0.5
        min_delta: 0.8
        max_delta: 1.2
    }
  }

  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 15000
            learning_rate: 0.00003
          }
          schedule {
            step: 20000
            learning_rate: 0.00001
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"
  num_steps: 25200
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}
train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!cp $ckpt_data $ckpt_data2
!cp $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import time

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)


import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

## V4

### Configuration

In [ ]:
# efficientdet_d0_coco17_tpu-32

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
%rm efficientdet_d0_coco17_tpu-32.tar.gz

%cd ..

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v4'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
# momentum_optimizer sembra essere il migliore in termini di loss
'''
model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
          height: 512
          width: 512
      }
    }

    feature_extractor {
      type: "ssd_efficientnet-b0_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 3
        num_filters: 64
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        use_dropout: true
        dropout_keep_probability: 0.8
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.9999998989515007e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: SWISH
          batch_norm {
            decay: 0.9900000095367432
            scale: true
            epsilon: 0.0010000000474974513
          }
          force_use_bias: true
        }
        depth: 64
        num_layers_before_predictor: 3
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 2.0
        aspect_ratios: [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.50, 2.75, 3.0]
        scales_per_octave: 3
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.1
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 1.5
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 2.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    add_background_class: false
  }
}

train_config {
#  freeze_variables: ["feature_extractor"]
  batch_size: 4

  data_augmentation_options {
    random_horizontal_flip {
        probability: 0.3
    }
    random_pixel_value_scale {
        minval: 0.9
        maxval: 1.2 
    }
    random_adjust_brightness {
        max_delta: 0.2
    }
    random_adjust_contrast {
        min_delta: 0.8
        max_delta: 1.2
    }
  }

  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: 0.003
          total_steps: 25200
          warmup_learning_rate: 0.001
          warmup_steps: 1000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"
  num_steps: 25200
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}
train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!cp $ckpt_data $ckpt_data2
!cp $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)


import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

# Faster R-CNN ResNet50 V1 640x640

## V1

### Configuration

In [ ]:
# faster_rcnn_resnet50_v1_640x640_coco17_tpu-8

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
%rm faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

%cd ..

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v1'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
'''
# Faster R-CNN with Resnet-50 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint

# Achieves -- mAP on COCO14 minival dataset.

# This config is TPU compatible.

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
          height: 640
          width: 640
      }
    }

    feature_extractor {
      type: 'faster_rcnn_resnet50_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0, 3.0]
        aspect_ratios: [0.5, 1.0, 1.5, 2.0, 2.50, 3.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 0.8
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 1.5
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config {
  batch_size: 2
  num_steps: 25200
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 13000
            learning_rate: 0.00003
          }
          schedule {
            step: 20000
            learning_rate: 0.000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  data_augmentation_options {
    random_horizontal_flip {
        probability: 0.5
    }
    random_pixel_value_scale {
        minval: 0.9
        maxval: 1.2
    }
    random_adjust_brightness {
        max_delta: 0.2
    }
    random_adjust_contrast {
        min_delta: 0.8
        max_delta: 1.2
    }
  }
  
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"

  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  use_bfloat16: false  # works only on TPUs
}

train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000

!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-51.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-51.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-51.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-51.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!mv $ckpt_data $ckpt_data2
!mv $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-51"), 'model_checkpoint_path: "{}"'.format("ckpt-51"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)

import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

## V2

### Configuration

In [ ]:
# faster_rcnn_resnet50_v1_640x640_coco17_tpu-8

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
%rm faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

%cd ..

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v2'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
'''
# Faster R-CNN with Resnet-50 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint

# Achieves -- mAP on COCO14 minival dataset.

# This config is TPU compatible.

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
          height: 640
          width: 640
      }
    }

    feature_extractor {
      type: 'faster_rcnn_resnet50_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0, 3.0]
        aspect_ratios: [0.5, 1.0, 1.5, 2.0]
        height_stride: 8
        width_stride: 8
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: true
        dropout_keep_probability: 0.8
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 1.5
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config: {
  batch_size: 2
#  freeze_variables: ["feature_extractor"]
  num_steps: 50400
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 25000
            learning_rate: 0.00003
          }
          schedule {
            step: 40000
            learning_rate: 0.000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"
  data_augmentation_options {
    random_horizontal_flip {
    }
    random_pixel_value_scale {

    }
    random_adjust_brightness {

    }
    random_adjust_contrast {
        
    }
  }

  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  use_bfloat16: false  # works only on TPUs
}

train_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000

!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-51.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-51.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-51.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-51.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!mv $ckpt_data $ckpt_data2
!mv $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-51"), 'model_checkpoint_path: "{}"'.format("ckpt-46"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="-1" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)

import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

## V3

### Configuration

In [ ]:
# faster_rcnn_resnet50_v1_640x640_coco17_tpu-8

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
%rm faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

%cd ..

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v3'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
'''
# Faster R-CNN with Resnet-50 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint

# Achieves -- mAP on COCO14 minival dataset.

# This config is TPU compatible.

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
          height: 640
          width: 640
      }
    }

    feature_extractor {
      type: 'faster_rcnn_resnet50_keras'
      batch_norm_trainable: true
    }

    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 3.9999998989515007e-05
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.2
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config: {
  batch_size: 2
  num_steps: 25200
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: 0.003
          total_steps: 25200
          warmup_learning_rate: 0.001
          warmup_steps: 1000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"

  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_brightness {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }

  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  use_bfloat16: false  # works only on TPUs
}

train_input_reader: {
  label_map_path: "/content/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}
'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path

In [ ]:
!cp /content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record /content/data/train.record

In [ ]:
!cp /content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt /content/data/label_map.pbtxt

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000

!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!cp $ckpt_data $ckpt_data2
!cp $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import time

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="-1" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)

import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)

In [ ]:
%matplotlib inline
eval_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'validation')):
    eval_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)

import random
start = random.randint(0,200)
end = start + 20
eval_samples = eval_samples[0:50]

inference_with_plot(eval_samples, draw_groundtruth=True, annotations_df=validation_annotations, box_th=0.30)

## V4

### Configuration

In [ ]:
# faster_rcnn_resnet50_v1_640x640_coco17_tpu-8

# ----------------------------------- BEGIN MODEL SETTINGS --------------------------------------- #
%cd $pre_trained_models_path
# SET DOWNLOAD LINK AND MODEL TECH NAME + .tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
# SET MODEL TECH NAME + .tar.gz
!tar -xf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
%rm faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

%cd ..

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v4'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(version_path):
    os.mkdir(version_path)
if not os.path.exists(version_path + "/eval"):
    os.mkdir(version_path + "/eval")
if not os.path.exists(version_path + "/train"):
    os.mkdir(version_path + "/train")
if not os.path.exists(version_path + "/custom_ckpt"):
    os.mkdir(version_path + "/custom_ckpt")

shutil.copy(pre_trained_conf_path, model_conf_path)

In [ ]:
# Modifiche manuali al config file:
'''
# Faster R-CNN with Resnet-50 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint

# Achieves -- mAP on COCO14 minival dataset.

# This config is TPU compatible.

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet50_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.2
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: true
        dropout_keep_probability: 0.8
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.2
        iou_threshold: 0.1
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config: {
  batch_size: 2
  num_steps: 25100
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.01
          schedule {
            step: 5000
            learning_rate: 0.001
          }
          schedule {
            step: 10000
            learning_rate: 0.0001
          }
          schedule {
            step: 15000
            learning_rate: 0.00001
          }
          schedule {
            step: 20000
            learning_rate: 0.000001
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }


  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"

  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  use_bfloat16: true  # works only on TPUs
}

train_input_reader: {
  label_map_path: "/content/data/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/data/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1
}

eval_input_reader: {
  label_map_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record"
  }
}

'''

In [ ]:
import re
test_conf_path = version_path + "/test.config"
with open(model_conf_path, 'r') as f:
    config = f.read()
with open(test_conf_path, 'w') as f:
    config = re.sub('(input_path: "/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record")', 'input_path: "{}"'.format(test_record_path), config)
    f.write(config)

### Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path

In [ ]:
!cp /content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record /content/data/train.record

In [ ]:
!cp /content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt /content/data/label_map.pbtxt

In [ ]:
# TRAINING
#checkpoint_every_n = int(num_train_samples / batch_size)
checkpoint_every_n = 1000

!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_every_n=$checkpoint_every_n \

In [ ]:
ckpt_path = version_path + "/checkpoint"
cust_ckpt_path = version_path + "/custom_ckpt"
ckpt_data = version_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx = version_path + "/ckpt-26.index"
ckpt_data2 = cust_ckpt_path + "/ckpt-26.data-00000-of-00001"
ckpt_idx2 = cust_ckpt_path + "/ckpt-26.index"
!cp $ckpt_path $cust_ckpt_path/checkpoint
!cp $ckpt_data $ckpt_data2
!cp $ckpt_idx $ckpt_idx2
import re
with open(cust_ckpt_path + "/checkpoint", 'r') as f:
    content = f.read()
with open(cust_ckpt_path + "/checkpoint", 'w') as f:
    content = re.sub('(model_checkpoint_path: "{}")'.format("ckpt-26"), 'model_checkpoint_path: "{}"'.format("ckpt-26"), content)
    f.write(content)

In [ ]:
# EXPORTER
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/exporter_main_v2.py \
  --pipeline_config_path=$model_conf_path \
  --trained_checkpoint_dir=$version_path/custom_ckpt \
  --output_directory=$exported_model_path \
  --input_type=image_tensor

### Testing

In [ ]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

%cd /content

import sys # importyng sys in order to access scripts located in a different folder

path2scripts = '/content/models/research' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

path2annotations = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'
train_annotations = pd.read_csv(os.path.join(path2annotations, 'train.csv'))
test_annotations = pd.read_csv(os.path.join(path2annotations, 'test.csv'))
validation_annotations = pd.read_csv(os.path.join(path2annotations, 'validation.csv'))

# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="-1" # specify your computational device


In [ ]:
path2config = exported_model_conf_path
path2model = exported_model_path + '/'

configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint/ckpt-0')).expect_partial()

path2label_map = label_map_path
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

In [ ]:
%matplotlib inline
train_samples = []
test_samples = []

for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'test')):
    test_samples.extend(os.path.join(path, name) for name in filenames)

#for (path, dirnames, filenames) in os.walk(os.path.join(images_path, 'train')):
#    train_samples.extend(os.path.join(path, name) for name in filenames)

import random
start = random.randint(0,200)
end = start + 20
train_samples = train_samples[start:end] # per testarne di meno di tutta la cartella
test_samples = test_samples[20:40]

inference_with_plot(test_samples, draw_groundtruth=True, annotations_df=test_annotations, box_th=0.30)